# Mini-project 1: Principal Component Analysis (PCA)
#### Author: Jace Kline

## Algorithm Development Steps

In this section, we walk through the steps behind implementing the PCA algorithm outlined in section 10.6 of the textbook. The algorithm will consist of four main steps:

1. Centering the original data set samples around the origin
2. Standardizing the data set based on standard deviation
3. Computing the eigendecomposition of the covariance matrix from the centered, standardized data set
4. Choosing the dimension for approximating the original space


#### Assumptions

* The data matrix X is structured such that rows are attributes and columns are samples
* The number of rows in data matrix X is less than the number of columns

In [307]:
# import libraries

import numpy as np
import pandas as pd
import sklearn.decomposition as skl

np.set_printoptions(suppress=True)

We will use the following data matrix below throughout the development of the algorithm to demonstrate intermediate steps.

In [308]:
# sample data matrix for testing/demonstration

A = np.array([
    [2,8,2,1,5],
    [8,7,2,2,6],
    [4,0,5,0,4]
])

print(A)
print(A.shape)

[[2 8 2 1 5]
 [8 7 2 2 6]
 [4 0 5 0 4]]
(3, 5)


### Step 1: Centering the Data Matrix

Per the algorithm for PCA in section 10.6 of the book, we must first center the data matrix so that each dimension has a mean of 0. We achieve this by computing the mean of each dimension (row) and then subtracting all elements in each row by the respective row's mean value.

In [309]:
D, samples = A.shape
rowmeans = np.mean(A, axis=1)
offsetmatrix = np.repeat(rowmeans, samples, axis=0).reshape((D,samples))
centered = A - offsetmatrix

print(centered)

[[-1.6  4.4 -1.6 -2.6  1.4]
 [ 3.   2.  -3.  -3.   1. ]
 [ 1.4 -2.6  2.4 -2.6  1.4]]


### Step 2: Standardization

The next step in the PCA algorithm involves standardizing each component of the data matrix by dividing by the component's respective standard deviation. We show this behavior below.

In [310]:
rowstds = np.std(centered, axis=1)
standardized = (centered.T / rowstds).T

print(standardized)

[[-0.62092042  1.70753116 -0.62092042 -1.00899568  0.54330537]
 [ 1.18585412  0.79056942 -1.18585412 -1.18585412  0.39528471]
 [ 0.64993368 -1.2070197   1.11417203 -1.2070197   0.64993368]]


### Step 3: Eigendecomposition of the Covariance Matrix

We must first find the covariance matrix of the centered and standardized data array. We then compute the eigendecomposition of this covariance matrix. Following this, we can choose a desired number of dimensions to use for dimensionality reduction.

In [311]:
covmatrix = np.cov(standardized)
print(covmatrix)

[[ 1.25        0.69030098 -0.39635072]
 [ 0.69030098  1.25        0.04587658]
 [-0.39635072  0.04587658  1.25      ]]


In [312]:
# Compute the eigenvalues and unit-length eigenvectors of the standardized covariance matrix
# The eigenvalues and vectors are ordered in ascending order

res = np.linalg.eigh(covmatrix)

# flip the order so the eigenvalues and vectors are sorted in descending order based on eigenvalue
eigvals = np.flip(res[0])
eigvecs = np.flip(res[1], axis=1)

print(eigvals)
print(eigvecs)

[2.026786  1.2895867 0.4336273]
[[ 0.71551818 -0.02918716 -0.69798413]
 [ 0.61644272  0.4964592   0.61116826]
 [-0.32868237  0.86756923 -0.3732178 ]]


We show below how we execute dimensionality reduction. We simply choose the same number of eigenvectors (descending order by weight) as the number dimensions we desire, and we represent these in a matrix as column vectors. We call this matrix B.

In [313]:
DESIRED_DIMENSIONS = 2

B = eigvecs[:, 0:DESIRED_DIMENSIONS]
print(B)

[[ 0.71551818 -0.02918716]
 [ 0.61644272  0.4964592 ]
 [-0.32868237  0.86756923]]


### Step 4: Projection

Using our dimension-reducing matrix, we can take vectors from the original space and project them onto the principal subspace with fewer dimensions than the original space. To express this projection in the original space, we multiply by the original standard deviation and add the mean of each for each vector component.

In [314]:
# x = a vector from the original space

x = np.array([1,1,1])


# x_standardized = x tranformed into centered, standardized space

x_standardized = (x - rowmeans) / rowstds


# x_principal = the vector obtained by transforming x_standardized into the principal subspace

x_principal = B @ (B.T @ x_standardized)


# x_approx = x_principal transformed back into the original space ~> An approximation of x after dimension reduction

x_approx = (x_principal * rowstds) + rowmeans

print("x =", x)
print("x_standardized =", x_standardized)
print("x_principal =", x_principal)
print("x_approx =", x_approx)

x = [1 1 1]
x_standardized = [-1.00899568 -1.58113883 -0.74278135]
x_principal = [-0.99842797 -1.59039212 -0.73713071]
x_approx = [1.02723108 0.97659083 1.01217185]


We see that in this example the approximation is quite close to the original sample after dimension reduction. 

## Algorithm Implementation

Now that we have illustrated the steps for the algorithm, we show the implementation of this procedure as a Python class.

In [315]:
# Python class to express the behavior of PCA analysis
class PCA:
    # initialize the PCA class with a given data set X (required)
    # optionally supply N, the number of reduction dimensions
    def __init__(self, X, M=1):
        # set the class variables
        self.X = X
        self.M = M
        D, samples = self.X.shape
        self.rowmeans = np.mean(X, axis=1)
        self.rowstds = np.std(self.X, axis=1)

        # standardize the centered data matrix
        self.standardized = self.standardize(X)

        # compute the covariance matrix
        self.covmatrix = np.cov(self.standardized)

        # compute the eigendecomposition of the covariance matrix
        res = np.linalg.eigh(self.covmatrix)
        self.eigvals = np.flip(res[0])
        self.eigvecs = np.flip(res[1], axis=1)

        # compute B
        self.B = self.eigvecs[:, 0:self.M]

    # set N, the number of dimensions to reduce to
    def set_M(self, M):
        # set N, recompute B
        self.M = M
        self.B = self.eigvecs[:, 0:self.M]

    # center and standardize variance to 1 of sample(s)
    def standardize(self, x):
        if len(x.shape) == 1: # 1d array
            return (x - self.rowmeans) / self.rowstds
        else:
            D, samples = x.shape
            centered = x - np.repeat(self.rowmeans, samples, axis=0).reshape((D,samples))
            standardized = (centered.T / self.rowstds).T
            return standardized

    # shift sample(s) back to original data space
    def unstandardize(self, x):
        if len(x.shape) == 1:
            return (x * self.rowstds) + self.rowmeans
        else:
            D, samples = x.shape
            return (x.T * self.rowstds).T + np.repeat(self.rowmeans, samples, axis=0).reshape((D,samples))

    # return the covariance matrix of the centered, standardized data
    def get_covariance_matrix(self):
        return self.covmatrix

    # transform a standardized sample of D dimensions into N dimensions
    def transform_reduce(self, x):
        return self.B.T @ x

    # transform a dimension-reduced sample of N dimensions into D dimensions
    # the result is centered and standardized
    def transform_inverse(self, z):
        return self.B @ z

    # perform end-to-end transformation
    # centers, standardizes, reduces, inverts, and unstandardizes
    # this function takes a sample and "approximates" it using PCA with given N
    def transform(self, x):
        x_standardized = self.standardize(x)
        x_principal = self.transform_inverse(self.transform_reduce(x_standardized))
        x_transformed = self.unstandardize(x_principal)
        return x_transformed

    # perform dimension reduction on the entire X data set
    # then map back to original X space
    def pca_reduce_dataset(self, M=-1):
        if M > 0:
            self.set_M(M)
        return self.unstandardize(self.B @ self.B.T @ self.standardized)

### Implementation Testing

Now that we have an implementation, we can test this implementation against the SciKit Learn implementation of PCA to ensure we get the same result on a test sample. The difference between our implementation and the SciKit Learn implementation is that our implementation automatically centers and standardizes the dataset prior to calculating the covariance matrix and eigendecompositions. Hence, we must have SciKit Learn's implementation perform PCA on the standardized data set, and we also must perform standardization on the sample point and undo this standardization after performing transformation. Below, we show the results of dimension reduction against sample x=(1,1,1) defined above. We see that both implementations compute the same covariance matrix and transformed sample.

In [316]:
# Our implementation of PCA dimension reduction

pca = PCA(A, M=DESIRED_DIMENSIONS)
print(pca.get_covariance_matrix())
print(pca.transform(x))

[[ 1.25        0.69030098 -0.39635072]
 [ 0.69030098  1.25        0.04587658]
 [-0.39635072  0.04587658  1.25      ]]
[1.02723108 0.97659083 1.01217185]


In [317]:
# SciKit Learn implementation of PCA dimension reduction
# This implementation does not automatically center and standardize the data set
# Hence, we must perform the PCA on the centered, standardized dataset

pca_skl = skl.PCA(n_components=DESIRED_DIMENSIONS, svd_solver='full')
pca_skl.fit(standardized.T)
print(pca_skl.get_covariance())
print(pca.unstandardize(pca_skl.inverse_transform(pca_skl.transform(pca.standardize(x).reshape(1,-1))).reshape(3,)))

[[ 1.25        0.69030098 -0.39635072]
 [ 0.69030098  1.25        0.04587658]
 [-0.39635072  0.04587658  1.25      ]]
[1.02723108 0.97659083 1.01217185]


## Analysis of Triathlon Data Set

We have obtained a data set from https://www.kaggle.com/mpwolke/wired-differently-triathlon/data. This data represents results from an Ironman 70.3 triathlon race that took place in 2019. We want to perform PCA analysis on the data set and compare PCA dimension reduction with SVD dimension reduction. We also want to explore the correlations between each of the sub-events (swim, bike, run) as well as overall time among the race participants.

### Data Cleaning

Prior to performing analysis, we first clean the data set into something useable in PCA dimension reduction. We ensure that all features of the data set are numerical and we remove unnecessary features not applicable to our analysis. Below we show the original data set compared to the cleaned data set. The data cleaning process can be found [here](./clean.ipynb).

In [318]:
# show the original data set
print(pd.read_csv("../data/original.csv").head())

# store the cleaned data set into a dataframe and print it
df = pd.read_csv("../data/cleaned.csv")
print(df.head())

   Unnamed: 0  Pos                       Name                     Club  \
0           1    1            FRODENO Jan (2)        Laz  Saarbruecken   
1           2    2        CLAVEL Maurice (24)                      NaN   
2           3    3  TAAGHOLT Miki  Morck (21)                    Ttsdu   
3           4    4         STRATMANN Jan (13)  Triathlon  Team  Witten   
4           5    5       MOLINARI Giulio (22)        C. S. Carabinieri   

        Cat        Swim          T1       Bike          T2         Run  \
0  MPRO - 1   00:22:501  00:01:2710  02:02:011  00:01:5221   01:11:252   
1  MPRO - 2  00:23:3311  00:01:3015  02:04:543   00:01:323   01:12:144   
2  MPRO - 3   00:22:574   00:01:141  02:05:526  00:02:0148   01:14:238   
3  MPRO - 4   00:22:502  00:01:3118  02:08:208  00:01:5633   01:13:336   
4  MPRO - 5   00:22:585  00:01:4968  02:05:114  00:02:0768  01:17:2117   

       Time  
0  03:39:35  
1  03:43:43  
2  03:46:27  
3  03:48:10  
4  03:49:26  
   Swim   T1  Bike   T2   

### PCA and SVD Dimension Reduction

We will start by performing PCA and SVD dimension reduction on each sample in our data set. We will vary the number of dimensions and observe the accuracy of each method. We use the SVD implementation provided by the SciKit Learn library, consistent with the SVD algorithm outlined in section 10.4 of the book.

In [319]:
# store the data set as an array
# features are rows, samples are columns
X = df.to_numpy().T
X_features, X_samples = X.shape

print("Data matrix:\n", X)
print("\nDimensions:", X.shape)

Data matrix:
 [[ 1370  1413  1377 ...  2481  2436  2230]
 [   87    90    74 ...   338   128   278]
 [ 7321  7494  7552 ...  9599  9676  9702]
 ...
 [13175 13423 13587 ... 19279 19286 19286]
 [    1     1     1 ...     1     1     1]
 [    9     9     9 ...     3     4     4]]

Dimensions: (8, 797)


In [320]:
# Create SVD decomposition of X
U, singular_values, Vt = np.linalg.svd(X, full_matrices=False, compute_uv=True)
Sigma = np.diag(singular_values)

# Create PCA decomposition of X
pca = PCA(X)

# define a function that computes the SVD approximation of X in N dimensions
def svd_reduce_dataset(M):
    return U[:,0:M] @ Sigma[0:M,0:M] @ Vt[0:M,:]

# define a function that computes the PCA approximation of X in N dimensions
def pca_reduce_dataset(M):
    return pca.pca_reduce_dataset(M=M)

# define a function that measures the addition of the sum of squares error for each sample between the original and approximate data matrix
def error_sum_of_squares(X_approx):
    return np.sum(np.sqrt(np.sum(np.square(X - X_approx), axis=0)))

In [321]:
# Loop over possible dimensions to reduce to and perform both SVD and PCA dimension reduction on X
# Compute the sum of squares value 
for M in range(1, X_features):
    svd_reduced = svd_reduce_dataset(M)
    pca_reduced = pca_reduce_dataset(M)
    
    svd_error = error_sum_of_squares(svd_reduced)
    pca_error = error_sum_of_squares(pca_reduced)

    print(f"M = {M}: SVD error = {svd_error}; PCA error = {pca_error}")

M = 1: SVD error = 315458.2170843122; PCA error = 408002.3055187013
M = 2: SVD error = 158226.04625871772; PCA error = 370857.82630964444
M = 3: SVD error = 47832.966943956155; PCA error = 366832.5805188172
M = 4: SVD error = 21862.17064038208; PCA error = 297928.13345273404
M = 5: SVD error = 1254.2185559626928; PCA error = 232482.10173219253
M = 6: SVD error = 222.92900976507406; PCA error = 195219.7247403326
M = 7: SVD error = 154.71946657773168; PCA error = 269.48678760111204


### Analysis of SVD vs PCA

With our given data set, it is evident by measuring the sum of the "distance" measures between the reduced matrices and the original matrix results in more accurate modeling by the SVD dimension reduction method over PCA. This can be explained through observation of the singular values and eigenvalues of the SVD and PCA methods, respectively. We see below that the singular values in the SVD decomposition have much greater percent differential between subsequent values, implying that the first few dimensions carry more "weight" in the decomposition, and therefore will keep the dimension reduction fairly accurate for reduced values of M. On the contrary, the PCA eigenvalues show relatively similar magnitudes, leading to the conclusion that the PCA is not finding highly weighted principals and therefore the dimension reduction approximation will suffer overall, particularly for lower values of M.

In [322]:
print("SVD singular values:\n", singular_values)
print("\nWeights of SVD singular values:\n", singular_values / np.sum(singular_values))

print("\nPCA eigenvalues:\n", pca.eigvals)
print("\nWeights of PCA eigenvalues:\n", pca.eigvals / np.sum(pca.eigvals))

SVD singular values:
 [588557.15869969  10892.83534843   6469.0560675    1812.64860764
   1106.68531687     56.92761533      7.63188929      5.62563419]

Weights of SVD singular values:
 [0.96657723 0.01788911 0.01062402 0.00297688 0.00181749 0.00009349
 0.00001253 0.00000924]

PCA eigenvalues:
 [3.75212819 1.12778623 0.94059361 0.73174192 0.63925611 0.47904664
 0.33949745 0.00000011]

Weights of PCA eigenvalues:
 [0.46842755 0.1407964  0.11742668 0.09135297 0.07980675 0.0598057
 0.04238393 0.00000001]
